# Transformação para formato Time Series

## Pacotes necessários

In [10]:
import pandas as pd
import fastparquet
import os

## Leitura dos dados



In [11]:
# Carregar o dataframe recodificado

# Costela
#df = pd.read_parquet("/home/costela/Documentos/Escorpiao-series/Dados_processados/df1_recodificado.parquet")

# Usuario
df = pd.read_parquet("/home/usuario/Documentos/Escorpiao_series/Dados_processados/2_df_recodificado.parquet")

# Filtrar os dados para incluir apenas os anos de 2014 a 2023
df = df[(df['ANO'] >= 2014) & (df['ANO'] <= 2023)]


## Agrupamento e calculo da Tx Incidencia

In [12]:
# Agrupar os dados por município, ano, mês e população
dfg = (
    df
    .groupby(['NOME_MUNI', 'ANO', 'MES', 'POPULACAO']) # Agrupa pelas variáveis
    .size() # Conta no número de registros por municipio, ano, mês e população
    .reset_index(name='CASOS') # transforma em dataframe e nomeia o número de contagens de registro como 'CASOS'
)

# Criar um dataframe por municipio

Este trecho tem dois objetivos principais: 
1. criar uma coluna de data padrão para representar cada mês e 
2. garantir que todos os municípios tenham uma sequência mensal completa no período observado, incluindo meses sem registro de casos.

## 1. Criar a coluna `DATA` (primeiro dia de cada mês)  
Como o banco possui `ANO` e `MES` separados, criamos uma data completa usando o primeiro dia do mês (`-01`). Isso facilita a ordenação, o uso de funções de série temporal e a criação de uma sequência mensal contínua.

- Convertemos `ANO` e `MES` para texto (`astype(str)`), concatenamos no formato `"YYYY-MM-01"` e transformamos para data com `pd.to_datetime()`.


In [15]:
# Criar coluna DATA (primeiro dia de cada mês)

dfg['DATA'] = pd.to_datetime(
    dict(
        year=dfg['ANO'].astype(int),
        month=dfg['MES'].astype(int),
        day=1
    )
)

## 2. Completar todos os meses para cada município  
Em seguida, criamos uma lista (`lista_municipios`) que armazenará um DataFrame “corrigido” para cada município. Para isso:

- Iteramos sobre o banco agrupado por município (`dfg.groupby('NOME_MUNI')`), trabalhando um município por vez.
- Ordenamos pelas datas para garantir que a série fique cronológica (`sort_values('DATA')`).
- Criamos uma sequência mensal completa com `pd.date_range()`, do menor ao maior mês disponível no município:
  - `freq='MS'` significa **Month Start**, isto é, sempre o **primeiro dia de cada mês**.
- Forçamos a existência de todos os meses usando `set_index('DATA')` + `reindex(datas_completas)`.  
  Isso insere linhas para os meses faltantes, preenchendo com `NaN` as variáveis que não existiam naquele mês.

Após o reindexamento, reconstituímos e padronizamos campos essenciais:

- Preenchemos o nome do município (`NOME_MUNI`) para as novas linhas inseridas.
- Recriamos `ANO` e `MES` a partir da coluna `DATA` (`dt.year` e `dt.month`).
- Preenchemos meses sem ocorrência com `CASOS = 0` (`fillna(0)`), assumindo ausência de registro como ausência de casos naquele mês.
- Preenchemos `POPULACAO` com o último valor conhecido, para manter a população constante por município (`ffill().bfill()`), garantindo preenchimento tanto para frente quanto para trás.

Por fim, cada DataFrame municipal é adicionado à lista `lista_municipios`, que depois pode ser concatenada para formar um banco final com séries mensais completas e comparáveis entre municípios.

In [16]:

# Completar todos os meses para cada município
lista_municipios = []

for municipio, df_mun in dfg.groupby('NOME_MUNI'):

    # Garantir ordenação
    df_mun = df_mun.sort_values('DATA')

    # Criar sequência mensal completa
    datas_completas = pd.date_range(
        start=df_mun['DATA'].min(),
        end=df_mun['DATA'].max(),
        freq='MS' # Month Start (primeiro dia de cada mes)
    )

    # Reindexar para forçar todos os meses
    df_mun = (
        df_mun
        .set_index('DATA')
        .reindex(datas_completas)
        .rename_axis('DATA')
        .reset_index()
    )

    # Preencher nome do município
    df_mun['NOME_MUNI'] = municipio

    # Recriar ANO e MES
    df_mun['ANO'] = df_mun['DATA'].dt.year
    df_mun['MES'] = df_mun['DATA'].dt.month

    # Preencher meses sem casos
    df_mun['CASOS'] = df_mun['CASOS'].fillna(0)

    # Preencher população (constante por município)
    df_mun['POPULACAO'] = df_mun['POPULACAO'].ffill().bfill()

    lista_municipios.append(df_mun)


## 3. Unir municipios em um dataframe e calcular taxas de incidência

É preciso unir tudo em um unico dataframe, porque os dataframes individuais dos municípios estão armazenados na lista "lista_municipios"

In [18]:
# Unir todos os municípios em um único DataFrame
df_mensal = pd.concat(lista_municipios, ignore_index=True) 

# Calcular taxa mensal por 100.000 habitantes
df_mensal['TAXA_MENSAL'] = (
    df_mensal['CASOS'] / df_mensal['POPULACAO']
) * 100000

# Arredonda taxa para decimal
df_mensal['TAXA_MENSAL'] = df_mensal['TAXA_MENSAL'].round(1)

# Ordena dataframes por nome do municipio e data
df_mensal = df_mensal.sort_values(['NOME_MUNI', 'DATA']).reset_index(drop=True)


## Salvar Dataframe

In [20]:
df_mensal.to_parquet(
    '/home/usuario/Documentos/Escorpiao_series/Dados_processados/3_df_agregado_muni.parquet',
    index=False
)                            